Notes:
- Table: 3 tables needed for the repors 
    - Ticket: main table and update frequently
    - Ticket metric: assocaite to Ticket and update frequently
    - Ticket form: assocaite to Ticket but do not change a lot
<br><br>
- Date: I use 20 days back on ticket created date as historical data might be updated. 
    - In the reports, we use ticket solved date not created date as filter, which means ticket created date could be way eariler.
    - I checked and found out 20 days covers almost everything and it takes about an hour for the whole process.
<br><br>
- Via: email and web only.
    - In the reports, we only need tickets via email and web (10% of all records).
    - Doing this can speed up the process.
    
Next Step:
- Load to S3
- Build dashboard in Tableau

In [1]:
import os
import json
import requests
import datetime
import pandas as pd
import numpy as np
from datetime import timedelta
from urllib.parse import urlencode

import boto3
from io import StringIO

In [2]:
# Credentials

user = os.environ.get('ZENDESK_USERNAME')
pwd = os.environ.get('ZENDESK_PASSWORD')

ticket_url = 'https://keyme.zendesk.com/api/v2/tickets' 
ticket_metric_url = 'https://keyme.zendesk.com/api/v2/ticket_forms'
ticket_form_url = 'https://keyme.zendesk.com/api/v2/tickets/metrics'

# print(user, pwd, ticket_url)

In [3]:
def get_ticket_data(ticket_url):
    
    df = pd.DataFrame(columns = ('ticket_id', 'ticket_via', 'ticket_created_at', 'ticket_form_id'))
    
    for i in range(1,21):
        
        yesterday = (datetime.date.today()-datetime.timedelta(i)).strftime("%Y-%m-%d")
        before_yesterday = (datetime.date.today()-datetime.timedelta(i+1)).strftime("%Y-%m-%d")
        
        local_yesterday = str(yesterday) +"T23:59:59Z"
        local_before_yesterday = str(before_yesterday) +"T23:59:59Z"
#         print('local_yesterday', local_yesterday)
#         print('local_before_yesterday', local_before_yesterday)
        
        params = {
        'query': 'created<'+local_yesterday+' type:ticket via:mail via:web',
        'sort_by': 'created_at',
        'sort_order': 'desc' 
        }   

        ticket_url = 'https://keyme.zendesk.com/api/v2/search.json?' + urlencode(params)    

        while ticket_url:
            response = requests.get(ticket_url, auth=(user, pwd))
            data = response.json()
            group_list = data['results'] 
            for group in group_list:
                if group['created_at'] >= local_before_yesterday:
                    df = df.append({'ticket_id': group['id'],
                                    'ticket_via': group['via']['channel'],
                                    'ticket_created_at': group['created_at'],
                                    'ticket_form_id': group['ticket_form_id']},
                                    ignore_index=True)

#                 print(group['created_at'], group['via'])

            if group['created_at'] <= local_before_yesterday: 
                break

            ticket_url = data['next_page']

    return df

In [4]:
# Tickets

ticket_data = get_ticket_data(ticket_url)

In [5]:
ticket_data.head()

,ticket_id,ticket_via,ticket_created_at,ticket_form_id
0,5535617,web,2021-06-22T23:55:27Z,360001003052
1,5535591,web,2021-06-22T23:51:32Z,360001003052
2,5535559,web,2021-06-22T23:46:42Z,360001003052
3,5535535,web,2021-06-22T23:42:56Z,360001003052
4,5535507,web,2021-06-22T23:39:11Z,360000055052


In [6]:
# Ticket metric

def get_ticket_metric_data(ticket_metric_url):
    
    ticket_id = [str(i) for i in ticket_data['ticket_id']]
    ticket_metric = pd.DataFrame(columns = ('metric_ticket_id', 'metric_created_at', 'metric_assigned_at', 'metric_solved_at'))

    for i in range(len(ticket_id)):

        ticket_metric_url = 'https://keyme.zendesk.com/api/v2/tickets/'+ticket_id[i]+'/metrics'
        response = requests.get(ticket_metric_url, auth=(user, pwd))
#         print(ticket_id[i])
        
        if response.status_code != 200:
            print('Status:', response.status_code, 'Problem with the request. Exiting.')
            exit()

        data = response.json()
        
        ticket_metric = ticket_metric.append({'metric_ticket_id': data['ticket_metric']['ticket_id'],
                                              'metric_created_at': data['ticket_metric']['created_at'], 
                                              'metric_assigned_at': data['ticket_metric']['assigned_at'],
                                              'metric_solved_at': data['ticket_metric']['solved_at'],
                                              'reply_time_in_minutes (calendar)': data['ticket_metric']['reply_time_in_minutes']['calendar'],
                                              'reply_time_in_minutes (business)': data['ticket_metric']['reply_time_in_minutes']['business'],
                                              'full_resolution_time_in_minutes (calendar)': data['ticket_metric']['full_resolution_time_in_minutes']['calendar'],
                                              'full_resolution_time_in_minutes (business)': data['ticket_metric']['full_resolution_time_in_minutes']['business']},
                                             ignore_index=True)

    return ticket_metric

In [7]:
ticket_metric_data = get_ticket_metric_data(ticket_metric_url)

In [8]:
ticket_metric_data.head()

,metric_ticket_id,metric_created_at,metric_assigned_at,metric_solved_at,full_resolution_time_in_minutes (business),full_resolution_time_in_minutes (calendar),reply_time_in_minutes (business),reply_time_in_minutes (calendar)
0,5535617,2021-06-22T23:55:28Z,2021-06-23T12:51:07Z,2021-06-23T12:51:07Z,0.0,776.0,0.0,776.0
1,5535591,2021-06-22T23:51:32Z,2021-06-23T12:44:52Z,2021-06-23T12:44:52Z,0.0,773.0,0.0,773.0
2,5535559,2021-06-22T23:46:42Z,None,None,NaN,NaN,0.0,780.0
3,5535535,2021-06-22T23:42:56Z,2021-06-23T12:44:12Z,2021-06-23T12:44:12Z,0.0,781.0,0.0,781.0
4,5535507,2021-06-22T23:39:12Z,None,None,NaN,NaN,0.0,785.0


In [9]:
# Ticket form

def get_ticket_form(ticket_form_url):
    
    ticket_form = pd.DataFrame(columns = ('form_id', 'form_created_at', 'form_name'))
    ticket_form_url = 'https://keyme.zendesk.com/api/v2/ticket_forms'
    response = requests.get(ticket_form_url, auth=(user, pwd))

    if response.status_code != 200:
        print('Status:', response.status_code, 'Problem with the request. Exiting.')
        exit()

    data = response.json()

    group_list = data['ticket_forms']
    for group in group_list:
        ticket_form = ticket_form.append({'form_id': group['id'], 
                                          'form_created_at': group['created_at'],
                                          'form_name': group['name']},
                                         ignore_index=True)
        
    return ticket_form

In [10]:
ticket_form_data = get_ticket_form(ticket_form_url)

In [11]:
ticket_and_metric = pd.merge(ticket_data, ticket_metric_data, how='left', left_on=['ticket_id'], right_on=['metric_ticket_id'])
final_combined_data = pd.merge(ticket_and_metric, ticket_form_data, how='left', left_on=['ticket_form_id'], right_on=['form_id'])

In [12]:
final_combined_data.shape

(7561, 15)

In [13]:
final_combined_data.head()

,ticket_id,ticket_via,ticket_created_at,ticket_form_id,metric_ticket_id,metric_created_at,metric_assigned_at,metric_solved_at,full_resolution_time_in_minutes (business),full_resolution_time_in_minutes (calendar),reply_time_in_minutes (business),reply_time_in_minutes (calendar),form_id,form_created_at,form_name
0,5535617,web,2021-06-22T23:55:27Z,360001003052,5535617,2021-06-22T23:55:28Z,2021-06-23T12:51:07Z,2021-06-23T12:51:07Z,0.0,776.0,0.0,776.0,360001003052,2020-07-16T15:37:14Z,Customer - Bad key
1,5535591,web,2021-06-22T23:51:32Z,360001003052,5535591,2021-06-22T23:51:32Z,2021-06-23T12:44:52Z,2021-06-23T12:44:52Z,0.0,773.0,0.0,773.0,360001003052,2020-07-16T15:37:14Z,Customer - Bad key
2,5535559,web,2021-06-22T23:46:42Z,360001003052,5535559,2021-06-22T23:46:42Z,None,None,NaN,NaN,0.0,780.0,360001003052,2020-07-16T15:37:14Z,Customer - Bad key
3,5535535,web,2021-06-22T23:42:56Z,360001003052,5535535,2021-06-22T23:42:56Z,2021-06-23T12:44:12Z,2021-06-23T12:44:12Z,0.0,781.0,0.0,781.0,360001003052,2020-07-16T15:37:14Z,Customer - Bad key
4,5535507,web,2021-06-22T23:39:11Z,360000055052,5535507,2021-06-22T23:39:12Z,None,None,NaN,NaN,0.0,785.0,360000055052,2018-05-07T19:10:03Z,General feedback


In [14]:
final_combined_data.to_csv('final_data.csv', index=False)